# Tutorial 2: Trap Deployment Records
## Filtering and Selecting Data with Pandas

---

### From the Field Notes of Gull, Storyteller and Trapper

*There were traps of three different sorts. One was long and narrow in simulation of a tunnel. Another was a box about three feet to a side. The last was spherical and made to hang from tree limbs.*

*Each of the traps looked as though it had been constructed by tossing scraps of wire and wood to an ape. None of the corners were square. Few of the edges were flush. Some of the traps appeared to have holes where the wire supports had been tugged and chewed at by some mad, starving carpenter's child.*

*"Is all this to trick them critters in?" one of the new men asked.*

*"No," I said. "If you know anything about carpentry or sculpture or even setting damn building blocks up in a tower, you know that making something shows the heart of the maker. And these traps here was the work of the Boss. She cobbled them together herself."*

---

## What You Will Learn

In this tutorial, you will learn to:

1. Filter rows using boolean conditions
2. Combine multiple conditions with `&` (and) and `|` (or)
3. Use `.loc[]` for label-based selection
4. Use `.isin()` for matching multiple values
5. Sort your data

By the end, you will be able to answer questions like:
- Which traps were deployed by the Redmane Expedition before the disaster?
- How many traps were successful vs. destroyed?
- What kinds of bait work for different creature categories?

---

In [ ]:
import pandas as pd

# Load the trap deployment records
traps = pd.read_csv('data/traps.csv')

# Also load creatures for reference
creatures = pd.read_csv('data/creatures.csv')

print(f"Trap records loaded: {len(traps)} deployments")
print(f"Creature catalog loaded: {len(creatures)} species")

In [ ]:
# First look at the trap data
traps.head(10)

Each row is a single trap deployment. The columns tell us:
- **trap_id**: Unique identifier
- **trap_type**: tunnel, box, sphere, pit, net, cage, snare, or hook
- **maker**: Who designed/built the trap
- **sector**: Where in the Quarry
- **deployment_date** / **retrieval_date**: When it was set and collected
- **condition**: good, worn, damaged, or destroyed
- **bait_type**: What was used to attract creatures
- **target_category**: What the trap was meant to catch
- **depth_placed_m**: How deep (0 = surface)
- **crew_id**: Which crew deployed it
- **successful**: Did it catch anything?
- **notes**: Field observations

---

## Part 1: Boolean Filtering

The most powerful way to select rows is by **boolean filtering**—creating a condition that is True or False for each row, then using that to select.

### Step 1: Create the condition

In [ ]:
# Which traps were successful?
traps['successful'] == True

This returns a Series of True/False values—one for each row. Rows where the trap caught something are True.

### Step 2: Use the condition to filter

In [ ]:
# Get only the successful traps
successful_traps = traps[traps['successful'] == True]
successful_traps

In [ ]:
# How many successful vs unsuccessful?
print(f"Successful traps: {len(successful_traps)}")
print(f"Failed traps: {len(traps) - len(successful_traps)}")
print(f"Success rate: {len(successful_traps) / len(traps) * 100:.1f}%")

About two-thirds of trap deployments catch something. Not bad, considering the conditions.

---

## Part 2: Comparison Operators

You can use all the standard comparison operators:

| Operator | Meaning |
|----------|--------|
| `==` | Equal to |
| `!=` | Not equal to |
| `>` | Greater than |
| `<` | Less than |
| `>=` | Greater than or equal |
| `<=` | Less than or equal |

In [ ]:
# Find traps deployed at depth (more than 0 meters)
deep_traps = traps[traps['depth_placed_m'] > 0]
print(f"Traps deployed underground: {len(deep_traps)}")
deep_traps

In [ ]:
# Find traps deployed at serious depth (40m or more)
# This is where the Maw Beast lives
very_deep = traps[traps['depth_placed_m'] >= 40]
print(f"Traps in the deep tunnels: {len(very_deep)}")
very_deep

Only the Deep Tunnel Syndicate (CRW006) works at these depths. Notice the trap conditions—"damaged" and "good." At 55 meters, you're in Maw Beast territory.

---

## Part 3: Filtering by Text Values

In [ ]:
# Find all traps made by The Boss
boss_traps = traps[traps['maker'] == 'The Boss']
print(f"The Boss made {len(boss_traps)} trap deployments")
boss_traps

The Boss designed three types: tunnel, box, and sphere. All deployed in Western Marsh or Cave Interior. Note trap TRP0005 and TRP0025—both destroyed in the Cave Interior during the expedition disaster.

And look at TRP0033: deployed March 15th, retrieved March 18th. That was the day of the attack.

In [ ]:
# Find all destroyed traps
destroyed = traps[traps['condition'] == 'destroyed']
print(f"Destroyed traps: {len(destroyed)}")
destroyed

Three destroyed traps total:
- TRP0005: Lost in the creature attack that killed Truck
- TRP0016: Burned by a wharver's hot belly on Grimslew Shore
- TRP0025: Lost in the same expedition disaster

The Quarry takes its toll on equipment. And on people.

---

## Part 4: Combining Conditions

Real questions often require multiple conditions. Use:
- `&` for AND (both conditions must be true)
- `|` for OR (either condition can be true)

**Important:** Each condition must be wrapped in parentheses!

In [ ]:
# Find successful traps in the Western Marsh
marsh_success = traps[(traps['sector'] == 'Western Marsh') & (traps['successful'] == True)]
print(f"Successful marsh traps: {len(marsh_success)}")
marsh_success

In [ ]:
# Find traps that are either destroyed OR damaged
bad_condition = traps[(traps['condition'] == 'destroyed') | (traps['condition'] == 'damaged')]
print(f"Traps in bad condition: {len(bad_condition)}")
bad_condition

In [ ]:
# Complex query: The Boss's traps that were successful AND in good/worn condition
boss_good = traps[
    (traps['maker'] == 'The Boss') & 
    (traps['successful'] == True) & 
    ((traps['condition'] == 'good') | (traps['condition'] == 'worn'))
]
boss_good

---

## Part 5: The `.isin()` Method

When you want to match multiple values, `.isin()` is cleaner than chaining OR conditions.

In [ ]:
# Find all traps of the three types The Boss designed
boss_types = traps[traps['trap_type'].isin(['tunnel', 'box', 'sphere'])]
print(f"Traps of Boss-style types: {len(boss_types)}")
boss_types['trap_type'].value_counts()

In [ ]:
# Find all traps from the crews that worked the Redmane expedition area
# CRW001 was Redmane, CRW002 is Gull's Remnants (formed after the disaster)
marsh_crews = traps[traps['crew_id'].isin(['CRW001', 'CRW002'])]
print(f"Traps from Redmane and Remnants: {len(marsh_crews)}")
marsh_crews

---

## Part 6: Negation with `~`

The tilde `~` negates a condition—gives you everything that does NOT match.

In [ ]:
# Find traps NOT made by Guild Standard design
non_guild = traps[~(traps['maker'] == 'Guild Standard')]
print(f"Non-guild traps: {len(non_guild)}")
non_guild['maker'].value_counts()

In [ ]:
# Find traps that did NOT target burrowing creatures
non_burrowing = traps[~traps['target_category'].isin(['burrowing'])]
non_burrowing['target_category'].value_counts()

---

## Part 7: Using `.loc[]` for Label-Based Selection

While `.iloc[]` selects by position, `.loc[]` selects by label (index value). It's also the proper way to select both rows AND columns at once.

In [ ]:
# Select specific columns for successful traps
traps.loc[traps['successful'] == True, ['trap_id', 'trap_type', 'crew_id', 'notes']]

In [ ]:
# Get just the essential info about deep tunnel traps
traps.loc[
    traps['depth_placed_m'] >= 20,
    ['trap_id', 'depth_placed_m', 'condition', 'successful', 'notes']
]

---

## Part 8: Sorting Data

Use `.sort_values()` to order your data.

In [ ]:
# Sort by depth, deepest first
traps.sort_values('depth_placed_m', ascending=False).head(10)

In [ ]:
# Sort by deployment date (chronological)
traps.sort_values('deployment_date').head(10)

In [ ]:
# Sort by multiple columns: first by crew, then by date
traps.sort_values(['crew_id', 'deployment_date']).head(15)

---

## Part 9: The Redmane Expedition Timeline

Let's use what we've learned to reconstruct the timeline of the Redmane Expedition's trap deployments.

In [ ]:
# Get all Redmane Expedition traps, sorted by date
redmane = traps[traps['crew_id'] == 'CRW001'].sort_values('deployment_date')
redmane[['trap_id', 'trap_type', 'deployment_date', 'retrieval_date', 'condition', 'successful', 'notes']]

The timeline tells a story:

- **March 12-15**: Initial surface deployments. Tunnel and sphere traps. Some success.
- **March 13-16**: More surface work. The expedition is getting established.
- **March 15-18**: Pre-expedition deployment. Setting up before going deep.
- **March 16-19**: More prep work.
- **March 17-18**: Cave interior deployment. Sphere trap—**destroyed**.
- **March 18**: Box trap goes into the cave. **Destroyed**. Same day as the sphere.

March 18th was the day. Two traps destroyed. Truck dead. The Boss missing. The expedition ended.

---

## Exercises

### Exercise 1: Grimslew Shore Operations

Find all traps deployed in the Grimslew Shore sector. How many were successful?

In [ ]:
# Your code here



### Exercise 2: Bait Analysis

Which bait types have been used? How many traps used each type?

*The Boss was known for her unconventional methods. Carrion for burrowers, seed paste for birds, but what did she use to lure the creature that killed Truck?*

In [ ]:
# Your code here



### Exercise 3: Trap Type Success Rates

For each trap type, find how many were successful and how many failed. Which trap type has the best success rate?

*Hint: You'll need to filter twice and compare, or use groupby (covered in Tutorial 4).*

In [ ]:
# Your code here
# Start by finding successful traps of each type



### Exercise 4: The Deep Tunnel Syndicate

Find all traps deployed by crew CRW006 (Deep Tunnel Syndicate) at depths of 40m or more. Sort by depth, deepest first.

*These are the people who brought back the Maw Beast carcass. The record price—950 units—for something that can digest anything, vegetal, ossified, or flesh.*

In [ ]:
# Your code here



### Exercise 5: Pre-Disaster vs Post-Disaster

The Redmane disaster occurred on March 18, 1855. Compare traps deployed before that date to those deployed after.

How many traps were deployed before March 18? After?

*Hint: You can compare date strings directly if they're in YYYY-MM-DD format.*

In [ ]:
# Your code here
# Try: traps[traps['deployment_date'] < '1855-03-18']



---

## Summary

In this tutorial, you learned:

| Concept | Code |
|---------|------|
| Filter by condition | `df[df['column'] == value]` |
| Comparison operators | `==`, `!=`, `>`, `<`, `>=`, `<=` |
| Combine with AND | `df[(condition1) & (condition2)]` |
| Combine with OR | `df[(condition1) \| (condition2)]` |
| Match multiple values | `df[df['column'].isin([val1, val2])]` |
| Negate a condition | `df[~condition]` |
| Select rows and columns | `df.loc[row_condition, [columns]]` |
| Sort by column | `df.sort_values('column')` |
| Sort descending | `df.sort_values('column', ascending=False)` |
| Sort by multiple | `df.sort_values(['col1', 'col2'])` |

---

## Next Tutorial

In **Tutorial 3: The Catch Ledger**, you will learn to sort, rank, and find extreme values in the expedition catch records. Which crews are catching the most? Which creatures fetch the highest prices? And what patterns emerge when you look at the data the way the Capital archivists do?

*"Most archivists live in the Capital, were born in the Capital, schooled in the Capital, work and will die in the Capital. But the very best of them come from outside Yeller Quarry."*

*Perhaps there's something in the Quarry that sharpens the mind. Or perhaps only a certain kind of mind survives there long enough to leave.*

---